# 1. Import Libraries

In [47]:
import pandas as pd
import numpy as np

In [48]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 2. Features to be Used

Below are the preliminary features that will be used to build the model. More will be added on afterwards.

**Fatigue**
- Days since last match

**Home Team Form**
- Goals difference of home team in the last x matches    
- Goals difference of home team in the last x home matches    
- Average number of points gained by home team in the last x matches
- Number of home matches won by home team in its last x home matches
- Home Team Win streak  
- Home Team Newly Promoted Team?

**Away Team Form**
- Goals difference of away team in the last x matches  
- Goals difference of away team in the last x away matches
- Average number of points gained by away team in the last x matches
- Number of away matches won by away team in its last x away matches
- Away Team Win streak
- Away Team Newly Promoted Team?

**Home Team Performance Index**
- Home Defense Performance Index
- Home Midfield Performance Index
- Home Attack Performance Index

**Away Team Performance Index**
- Away Defense Performance Index
- Away Midfield Performance Index
- Away Attack Performance Index

**Betting Odds**
- B365H
- B365D
- B365A

# 3. Data Preprocessing

There are two main datasets (Dataset 1 and Dataset 2) for each season that will be used to extract the features needed for the model. 

First we create an empty DataFrame. This DataFrame will eventually contain the data integrated from the two datasets.

In [49]:
df = pd.DataFrame()

In [50]:
seasons = ['2012-2013', '2013-2014', '2014-2015', '2015-2016', '2016-2017', 
           '2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022']

## 3.1 Dataset 1

Data Source: www.football-data-co.uk

Firstly we will concatenate every Dataset 1 for every season into a single DataFrame. Then we will do some data preprocessing steps on the DataFrame.

In [51]:
# standardize the teams names across all datasets

rename_teams = {'Arsenal': 'arsenal', 'Brighton': 'brighton', 'Chelsea': 'chelsea', 'Crystal Palace': 'palace', 'Everton': 'everton', 
                'Southampton': 'southampton', 'Watford': 'watford', 'West Brom': 'west-brom', 'Man United': 'united', 'Newcastle': 'newcastle',
                'Bournemouth': 'bournemouth', 'Burnley': 'burnley', 'Leicester': 'leicester', 'Liverpool': 'liverpool', 'Stoke': 'stoke',
                'Swansea': 'swansea', 'Huddersfield': 'huddersfield', 'Tottenham': 'tottenham', 'Man City': 'city', 'West Ham': 'west-ham',
                'Fulham': 'fulham', 'Wolves': 'wolves', 'Cardiff': 'cardiff', 'Aston Villa': 'aston-villa', 'Norwich': 'norwich',
                'Sheffield United': 'sheffield', 'Leeds': 'leeds', 'Brentford':'brentford', 'QPR': 'qpr', 'Reading': 'reading',
                'Wigan': 'wigan', 'Sunderland': 'sunderland', 'Hull': 'hull', 'Middlesbrough': 'middlesbrough'}

In [52]:
dataset1_df = pd.DataFrame()

for season in seasons:
    # read csv file for match statistics
    temp_df = pd.read_csv(f'datasets/data-source-1/{season}/dataset1.csv')
    
    # remove rows with no HomeTeam information
    temp_df = temp_df[temp_df['HomeTeam'].notna()]
    
    # rename team names in the 'HomeTeam' and 'AwayTeam' columns for standardized team names
    temp_df['HomeTeam'] = temp_df['HomeTeam'].apply(lambda word : rename_teams[word])
    temp_df['AwayTeam'] = temp_df['AwayTeam'].apply(lambda word : rename_teams[word])
    
    # concatenate temp_df to dataset1_df
    if dataset1_df.empty:
        dataset1_df = temp_df
    else:
        dataset1_df = pd.concat([dataset1_df, temp_df]).reset_index(drop=True)

In [53]:
# Make sure we have 5 seasons x 380 matches = 1900 matches in the DataFrame
dataset1_df.shape

(3800, 21)

In [54]:
dataset1_df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A
0,18/08/12,arsenal,sunderland,0.0,0.0,D,14.0,3.0,4.0,2.0,12.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.40,4.50,8.50
1,18/08/12,fulham,norwich,5.0,0.0,H,11.0,4.0,9.0,2.0,12.0,11.0,6.0,3.0,0.0,0.0,0.0,0.0,1.80,3.60,4.50
2,18/08/12,newcastle,tottenham,2.0,1.0,H,6.0,12.0,4.0,6.0,12.0,8.0,3.0,5.0,2.0,2.0,0.0,0.0,2.50,3.40,2.75
3,18/08/12,qpr,swansea,0.0,5.0,A,20.0,12.0,11.0,8.0,11.0,14.0,5.0,3.0,2.0,2.0,0.0,0.0,2.00,3.40,3.80
4,18/08/12,reading,stoke,1.0,1.0,D,9.0,6.0,3.0,3.0,9.0,14.0,4.0,3.0,2.0,4.0,0.0,1.0,2.38,3.25,3.10


### 3.1.1 Converting Data Types

In [55]:
dataset1_df.dtypes

Date         object
HomeTeam     object
AwayTeam     object
FTHG        float64
FTAG        float64
FTR          object
HS          float64
AS          float64
HST         float64
AST         float64
HF          float64
AF          float64
HC          float64
AC          float64
HY          float64
AY          float64
HR          float64
AR          float64
B365H       float64
B365D       float64
B365A       float64
dtype: object

In [56]:
def standardizeDate(date):
    year = date.split('/')[-1]
    
    if len(year) == 2:
        date = pd.to_datetime(date, format="%d/%m/%y")
    elif len(year) == 4:
        date = pd.to_datetime(date, format="%d/%m/%Y")
        
    return date

# convert 'Date' column to datetime object
dataset1_df['Date'] = dataset1_df['Date'].apply(lambda x: standardizeDate(x))
dataset1_df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A
0,2012-08-18,arsenal,sunderland,0.0,0.0,D,14.0,3.0,4.0,2.0,12.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.40,4.50,8.50
1,2012-08-18,fulham,norwich,5.0,0.0,H,11.0,4.0,9.0,2.0,12.0,11.0,6.0,3.0,0.0,0.0,0.0,0.0,1.80,3.60,4.50
2,2012-08-18,newcastle,tottenham,2.0,1.0,H,6.0,12.0,4.0,6.0,12.0,8.0,3.0,5.0,2.0,2.0,0.0,0.0,2.50,3.40,2.75
3,2012-08-18,qpr,swansea,0.0,5.0,A,20.0,12.0,11.0,8.0,11.0,14.0,5.0,3.0,2.0,2.0,0.0,0.0,2.00,3.40,3.80
4,2012-08-18,reading,stoke,1.0,1.0,D,9.0,6.0,3.0,3.0,9.0,14.0,4.0,3.0,2.0,4.0,0.0,1.0,2.38,3.25,3.10


### 3.1.2 Checking for Missing Values

In [57]:
dataset1_df.isnull().sum()

Date        0
HomeTeam    0
AwayTeam    0
FTHG        0
FTAG        0
FTR         0
HS          0
AS          0
HST         0
AST         0
HF          0
AF          0
HC          0
AC          0
HY          0
AY          0
HR          0
AR          0
B365H       0
B365D       0
B365A       0
dtype: int64

### 3.1.3 Data Transformation

In [58]:
dataset1_df['FTR'].unique()

array(['D', 'H', 'A'], dtype=object)

In [59]:
def transform_FTR(word):
    if word == 'H':
        return 'W'
    elif word == 'A':
        return 'L'
    elif word == 'D':
        return 'D'

dataset1_df['FTR'] = dataset1_df['FTR'].apply(lambda x: transform_FTR(x))
dataset1_df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A
0,2012-08-18,arsenal,sunderland,0.0,0.0,D,14.0,3.0,4.0,2.0,12.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.40,4.50,8.50
1,2012-08-18,fulham,norwich,5.0,0.0,W,11.0,4.0,9.0,2.0,12.0,11.0,6.0,3.0,0.0,0.0,0.0,0.0,1.80,3.60,4.50
2,2012-08-18,newcastle,tottenham,2.0,1.0,W,6.0,12.0,4.0,6.0,12.0,8.0,3.0,5.0,2.0,2.0,0.0,0.0,2.50,3.40,2.75
3,2012-08-18,qpr,swansea,0.0,5.0,L,20.0,12.0,11.0,8.0,11.0,14.0,5.0,3.0,2.0,2.0,0.0,0.0,2.00,3.40,3.80
4,2012-08-18,reading,stoke,1.0,1.0,D,9.0,6.0,3.0,3.0,9.0,14.0,4.0,3.0,2.0,4.0,0.0,1.0,2.38,3.25,3.10


## 3.2 Dataset 2

Data Source: www.fbref.com/en

Firstly we will concatenate every Dataset 2 for every season into a single DataFrame. Then we will do some data preprocessing steps on the DataFrame.

In [77]:
# standardize the teams names across all datasets

rename_teams = {'Leicester City':'leicester', 'Bournemouth':'bournemouth', 'West Brom':'west-brom', 'Brighton': 'brighton', 'Swansea City':'swansea', 
                'Tottenham':'tottenham', 'Huddersfield':'huddersfield', 'Manchester Utd': 'united', 'Newcastle Utd':'newcastle', 'Liverpool':'liverpool', 
                'Chelsea': 'chelsea', 'Crystal Palace': 'palace', 'Everton': 'everton', 'Manchester City':'city', 'Watford': 'watford', 
                'Stoke City':'stoke', 'Southampton': 'southampton', 'West Ham':'west-ham', 'Burnley':'burnley', 'Arsenal': 'arsenal', 
                'Wolves':'wolves', 'Fulham':'fulham', 'Cardiff City':'cardiff', 'Aston Villa':'aston-villa', 'Sheffield Utd':'sheffield',
                'Norwich City':'norwich', 'Leeds United':'leeds', 'Brentford':'brentford', 'Sunderland': 'sunderland', 'QPR': 'qpr',
                'Reading': 'reading', 'Wigan Athletic': 'wigan', 'Hull City': 'hull', 'Middlesbrough': 'middlesbrough'}

### 3.2.1 Feature Engineering

While doing data preprocessing for Dataset 2, we will also create a new feature **HDaysLastPlayed** and **ADaysLastPlayed** along the way.

The **HDaysLastPlayed** feature indicates the number of days since the home team's last match.

The **ADaysLastPlayed** feature indicates the number of days since the away team's last match.

Supposedly, we should create new features in the Feature Engineering step later. However, these two features must be created using information obtained from the raw datasets. It is not possible to create these two features after we have concatenated every Dataset 2 of each season into a single dataframe. Therefore, we will create these two features before we concatenate every Dataset 2 of each season into a single dataframe.

In [78]:
# returns the number of days since home team's last match
def getHDaysLastPlayed(row):
    HDaysLastPlayed = str(row['DaysLastPlayed']).split()[0]
    return HDaysLastPlayed

In [79]:
# returns the number of days since away team's last match
def getADaysLastPlayed(row):
    
    date = row['Date']
    team = row['team']
    opponent = row['Opponent']
    
    filter_condition = (concatenated_df['Date'] == date) & (concatenated_df['team'] == opponent) & (concatenated_df['Opponent'] == team)
    ADaysLastPlayed = str(concatenated_df[filter_condition]['DaysLastPlayed']).split()[1]    
    return ADaysLastPlayed

In [80]:
dataset2_df = pd.DataFrame()

for season in seasons:
    
    concatenated_df = pd.read_csv(f'datasets/data-source-2/{season}-data-source-2.csv')
  
     # convert 'Date' column to datetime object
    concatenated_df['Date'] =  pd.to_datetime(concatenated_df['Date'], format="%Y/%m/%d")
    
    # get DaysLastPlayed for all matches
    concatenated_df['DaysLastPlayed'] = concatenated_df['Date'] - concatenated_df['Date'].shift(1)
    
    # filter by Premier League matches only
    concatenated_df = concatenated_df[concatenated_df['Comp'] == 'Premier League']
    
    # rename team names in the 'Opponent' column for standardized team names
    concatenated_df['Opponent'] = concatenated_df['Opponent'].apply(lambda word : rename_teams[word])
    
    # add a new feature: HDaysLastPlayed (number of days since home team's last match)
    concatenated_df['HDaysLastPlayed'] = concatenated_df.apply(lambda row: getHDaysLastPlayed(row), axis=1)
    
    # add a new feature: ADaysLastPlayed (number of days since away team's last match)
    concatenated_df['ADaysLastPlayed'] = concatenated_df.apply(lambda row: getADaysLastPlayed(row), axis=1)
    
    # filter by home matches only
    concatenated_df = concatenated_df[concatenated_df['Venue'] == 'Home'].reset_index(drop=True)
    
    # drop 'Comp' and 'Venue' column
    concatenated_df.drop(['Comp', 'Venue', 'DaysLastPlayed'], axis=1, inplace = True)
        
    # rename features
    concatenated_df = concatenated_df.rename(columns={'xG': 'HxG', 'xGA': 'AxG', 'Poss': 'HPoss', 'Opponent': 'AwayTeam', 'team': 'HomeTeam'})
    
    if dataset2_df.empty:
        dataset2_df = concatenated_df
    else:
        dataset2_df = pd.concat([dataset2_df, concatenated_df]).reset_index(drop=True)

In [81]:
# Make sure we have 5 x 380 = 1900 matches in the DataFrame
dataset2_df.shape

(3800, 11)

In [82]:
dataset2_df.head()

,Date,Result,GF,GA,AwayTeam,HPoss,HomeTeam,HDaysLastPlayed,ADaysLastPlayed,HxG,AxG
0,2012-08-18,D,0,0,sunderland,NaN,arsenal,NaT,-274,NaN,NaN
1,2012-09-15,W,6,1,southampton,NaN,arsenal,13,13,NaN,NaN
2,2012-09-29,L,1,2,chelsea,NaN,arsenal,6,7,NaN,NaN
3,2012-10-27,W,1,0,qpr,NaN,arsenal,3,6,NaN,NaN
4,2012-11-10,D,3,3,fulham,NaN,arsenal,4,7,NaN,NaN


### 3.2.2 Converting Data Types

In [83]:
dataset2_df.dtypes

Date               datetime64[ns]
Result                     object
GF                         object
GA                         object
AwayTeam                   object
HPoss                     float64
HomeTeam                   object
HDaysLastPlayed            object
ADaysLastPlayed            object
HxG                       float64
AxG                       float64
dtype: object

In [84]:
dataset2_df['GF'] = pd.to_numeric(dataset2_df['GF'])
dataset2_df['GA'] = pd.to_numeric(dataset2_df['GA'])
dataset2_df.head()

,Date,Result,GF,GA,AwayTeam,HPoss,HomeTeam,HDaysLastPlayed,ADaysLastPlayed,HxG,AxG
0,2012-08-18,D,0,0,sunderland,NaN,arsenal,NaT,-274,NaN,NaN
1,2012-09-15,W,6,1,southampton,NaN,arsenal,13,13,NaN,NaN
2,2012-09-29,L,1,2,chelsea,NaN,arsenal,6,7,NaN,NaN
3,2012-10-27,W,1,0,qpr,NaN,arsenal,3,6,NaN,NaN
4,2012-11-10,D,3,3,fulham,NaN,arsenal,4,7,NaN,NaN


### 3.2.3 Checking for Missing Values

In [85]:
dataset2_df.isnull().sum()

Date                  0
Result                0
GF                    0
GA                    0
AwayTeam              0
HPoss               760
HomeTeam              0
HDaysLastPlayed       0
ADaysLastPlayed       0
HxG                1900
AxG                1900
dtype: int64

### 3.2.4 Data Transformation

In [86]:
dataset2_df['HDaysLastPlayed'].unique()

array(['NaT', '13', '6', '3', '4', '7', '12', '14', '8', '10', '11', '9',
       '15', '5', '-273', '-272', '16', '-274', '25', '21', '2', '-266',
       '-265', '-267', '-285', '-281', '-280', '-283', '-279', '-286',
       '-287', '17', '18', '-291', '22', '-294', '-295', '100', '-351',
       '105', '101', '-350', '106', '104', '-372', '-251', '-253', '-252',
       '-246', '-255', '-282'], dtype=object)

In [87]:
dataset2_df['ADaysLastPlayed'].unique()

array(['-274', '13', '7', '6', '3', '12', '4', '11', '14', '5', '8', '10',
       '15', '9', '2', '-273', '-272', '16', '21', '-267', '-266', '-265',
       '19', '-281', '-279', '-280', '-287', '18', '-284', '-275', '17',
       '22', '102', '105', '-351', '107', '-352', 'NaT', '104', '-371',
       '-357', '99', '-350', '103', '100', '-251', '23', '-253', '-258',
       '-252', '-250', '20'], dtype=object)

In [88]:
def transform_DaysLastPlayed(word):
    if word == 'NaT':
        return 7
    
    if int(word) < 0 or int(word) >= 10:
        return 10
    else:
        return int(word)

dataset2_df['HDaysLastPlayed'] = dataset2_df['HDaysLastPlayed'].apply(lambda x: transform_DaysLastPlayed(x))
dataset2_df['ADaysLastPlayed'] = dataset2_df['ADaysLastPlayed'].apply(lambda x: transform_DaysLastPlayed(x))
dataset2_df.head()

,Date,Result,GF,GA,AwayTeam,HPoss,HomeTeam,HDaysLastPlayed,ADaysLastPlayed,HxG,AxG
0,2012-08-18,D,0,0,sunderland,NaN,arsenal,7,10,NaN,NaN
1,2012-09-15,W,6,1,southampton,NaN,arsenal,10,10,NaN,NaN
2,2012-09-29,L,1,2,chelsea,NaN,arsenal,6,7,NaN,NaN
3,2012-10-27,W,1,0,qpr,NaN,arsenal,3,6,NaN,NaN
4,2012-11-10,D,3,3,fulham,NaN,arsenal,4,7,NaN,NaN


## 3.3 Dataset 3

In [90]:
# standardize the teams names across all datasets

rename_teams = {'Liverpool': 'liverpool', 'Manchester City': 'city', 'Chelsea': 'chelsea', 'Manchester United': 'united', 'Tottenham Hotspur': 'tottenham', 
                'Leicester City': 'leicester', 'Arsenal': 'arsenal', 'West Ham United': 'west-ham', 'Aston Villa': 'aston-villa', 'Wolverhampton Wanderers': 'wolves',
                'Everton': 'everton', 'Newcastle United': 'newcastle', 'Leeds United': 'leeds', 'Crystal Palace': 'palace', 'Burnley': 'burnley', 
                'Southampton': 'southampton', 'Brighton & Hove Albion': 'brighton', 'Brentford': 'brentford', 'Watford': 'watford', 'Norwich City': 'norwich', 
                'Fulham': 'fulham', 'West Bromwich Albion': 'west-brom', 'Sheffield United': 'sheffield', 'AFC Bournemouth': 'bournemouth', 'Bournemouth': 'bournemouth', 
                'Huddersfield Town': 'huddersfield', 'Cardiff City': 'cardiff', 'Stoke City': 'stoke', 'Swansea City': 'swansea', 'Sunderland': 'sunderland', 
                'Hull City': 'hull', 'Middlesbrough': 'middlesbrough', 'Queens Park Rangers': 'qpr', 'Wigan Athletic': 'wigan', 'Reading FC': 'reading',
                'Reading': 'reading', 'Blackburn Rovers': 'blackburn', 'Bolton Wanderers': 'bolton', 'Birmingham City': 'birmingham',
                'Blackpool':'blackpool', 'Portsmouth': 'portsmouth', 'West Bromwich':'west-brom', 'Chelsea FC': 'chelsea', 
                'Arsenal FC':'arsenal', 'Derby County': 'derby', 'Charlton Athletic': 'charlton', 'Nottingham Forest': 'nottingham', 
                'Coventry City': 'coventry', 'Millwall':'millwall', 'Preston North End': 'preston', 'Luton Town': 'luton', 
                'Bristol City': 'bristol', 'Barnsley': 'barnsley', 'Peterborough United': 'peterborough', 'Sheffield Wednesday': 'wednesday', 
                'Rotherham United': 'rotherham', 'Wycombe Wanderers': 'wycombe', 'Ipswich Town': 'ipswich', 'Burton Albion': 'burton',
                'Milton Keynes Dons': 'milton', 'Doncaster Rovers': 'doncaster', 'Yeovil Town': 'yeovil', 'Scunthorpe United':'scunthorpe', 'Plymouth Argyle': 'plymouth', 
                'Wolverhampton':'wolves', 'Colchester United': 'colchester', 'Southend United': 'southend', 'Brighton Hove Albion':'brighton',
                'Crewe':'crewe', 'Nottm Forest':'nottingham', 'Rotherham':'rotherham', 'Gillingham':'gillingham'}

In [91]:
dataset3_df = pd.read_csv(f'datasets/data-source-3/dataset3.csv')

# remove irrelevant data
teams = list(rename_teams.keys())
dataset3_df = dataset3_df[dataset3_df['Team'].isin(teams)]

# rename team names in the 'Team' column for standardized team names
dataset3_df['Team'] = dataset3_df['Team'].apply(lambda word : rename_teams[word])

dataset3_df.head()

,Version,Date,Team,Attack,Midfield,Defense
0,FIFA 22,"Aug. 18, 2022",liverpool,86,84,85
1,FIFA 22,"Aug. 18, 2022",city,84,87,86
2,FIFA 22,"Aug. 18, 2022",chelsea,84,85,83
3,FIFA 22,"Aug. 18, 2022",united,82,83,81
4,FIFA 22,"Aug. 18, 2022",tottenham,83,81,78


In [92]:
dataset3_df.shape

(28406, 6)

### 3.3.2 Checking for Missing Values

In [93]:
dataset3_df.isnull().sum()

Version     0
Date        0
Team        0
Attack      0
Midfield    0
Defense     0
dtype: int64

### 3.3.3. Converting Data Types 

In [94]:
dataset3_df.dtypes

Version     object
Date        object
Team        object
Attack       int64
Midfield     int64
Defense      int64
dtype: object

In [95]:
def refactor_date(date):
    months_dict = {'Jan': '01', 'Feb': '02', 'March': '03', 'April': '04', 'May': '05', 'June': '06', 
                   'July': '07', 'Aug': '08', 'Sept': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
    
    date = date.replace(',', '')
    date = date.replace('.', '')
    
    # replace first word of month 
    date = date.replace(date.split()[0], months_dict[date.split()[0]])
    date = date.replace(' ', '/')
    
    return date

In [96]:
dataset3_df['Date'] = dataset3_df['Date'].apply(lambda date: refactor_date(date))
dataset3_df['Date'] = pd.to_datetime(dataset3_df['Date'], format="%m/%d/%Y")
dataset3_df.head()

,Version,Date,Team,Attack,Midfield,Defense
0,FIFA 22,2022-08-18,liverpool,86,84,85
1,FIFA 22,2022-08-18,city,84,87,86
2,FIFA 22,2022-08-18,chelsea,84,85,83
3,FIFA 22,2022-08-18,united,82,83,81
4,FIFA 22,2022-08-18,tottenham,83,81,78


In [97]:
dataset3_df.to_csv(f'datasets/data-source-3/cleaned_dataset3.csv', index=False)

# 4. Integration of Datasets

## 4.1 Integrating Dataset 1 and Dataset 2

In [98]:
df

""


In [99]:
# merge two data sources into one DataFrame

df = dataset1_df
df = pd.merge(df, dataset2_df, on=['Date', 'HomeTeam', 'AwayTeam'])

In [100]:
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,GF,GA,HPoss,HDaysLastPlayed,ADaysLastPlayed,HxG,AxG
0,2012-08-18,arsenal,sunderland,0.0,0.0,D,14.0,3.0,4.0,2.0,12.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.40,4.50,8.50,D,0,0,NaN,7,10,NaN,NaN
1,2012-08-18,fulham,norwich,5.0,0.0,W,11.0,4.0,9.0,2.0,12.0,11.0,6.0,3.0,0.0,0.0,0.0,0.0,1.80,3.60,4.50,W,5,0,NaN,10,10,NaN,NaN
2,2012-08-18,newcastle,tottenham,2.0,1.0,W,6.0,12.0,4.0,6.0,12.0,8.0,3.0,5.0,2.0,2.0,0.0,0.0,2.50,3.40,2.75,W,2,1,NaN,10,10,NaN,NaN
3,2012-08-18,qpr,swansea,0.0,5.0,L,20.0,12.0,11.0,8.0,11.0,14.0,5.0,3.0,2.0,2.0,0.0,0.0,2.00,3.40,3.80,L,0,5,NaN,10,10,NaN,NaN
4,2012-08-18,reading,stoke,1.0,1.0,D,9.0,6.0,3.0,3.0,9.0,14.0,4.0,3.0,2.0,4.0,0.0,1.0,2.38,3.25,3.10,D,1,1,NaN,10,10,NaN,NaN


In [101]:
df.shape

(3800, 29)

## 4.2 Dropping Redundant Columns

In [102]:
df.drop(['FTR', 'GF', 'GA'], axis=1, inplace = True)
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HPoss,HDaysLastPlayed,ADaysLastPlayed,HxG,AxG
0,2012-08-18,arsenal,sunderland,0.0,0.0,14.0,3.0,4.0,2.0,12.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.40,4.50,8.50,D,NaN,7,10,NaN,NaN
1,2012-08-18,fulham,norwich,5.0,0.0,11.0,4.0,9.0,2.0,12.0,11.0,6.0,3.0,0.0,0.0,0.0,0.0,1.80,3.60,4.50,W,NaN,10,10,NaN,NaN
2,2012-08-18,newcastle,tottenham,2.0,1.0,6.0,12.0,4.0,6.0,12.0,8.0,3.0,5.0,2.0,2.0,0.0,0.0,2.50,3.40,2.75,W,NaN,10,10,NaN,NaN
3,2012-08-18,qpr,swansea,0.0,5.0,20.0,12.0,11.0,8.0,11.0,14.0,5.0,3.0,2.0,2.0,0.0,0.0,2.00,3.40,3.80,L,NaN,10,10,NaN,NaN
4,2012-08-18,reading,stoke,1.0,1.0,9.0,6.0,3.0,3.0,9.0,14.0,4.0,3.0,2.0,4.0,0.0,1.0,2.38,3.25,3.10,D,NaN,10,10,NaN,NaN


In [103]:
df.to_csv('datasets/dataset.csv', index=False)